In [ ]:
import os
import re
import time
import collections
import numpy as np
import pandas as pd
import spacy
import wordninja
from wikipedia2vec import Wikipedia2Vec
import pymysql

In [ ]:
# get method names for trainging
self_compiled_method_names = []
full2simple = {}
with open('/home/qwe/disk1/data_SoC/files_20w/simple_names.txt', 'r') as f:
    for i, line in enumerate(f):
        full = line.split(':::')[0]
        simple = line.split(':::')[-1].strip()
        full2simple.setdefault(full, None)
        full2simple[full] = simple
        self_compiled_method_names.append(simple)
print(len(self_compiled_method_names))
print(len(set(self_compiled_method_names)))
print(len(full2simple))

In [ ]:
# get method names for transfer learning
third_method_names_dictionary = {}
third_method_names = []
with open('/home/qwe/disk1/sources/TrainingMethodInfo.txt', 'r') as f:
    for line in enumerate(f):
        name = line[1].split(':')[3]
        tokens = line[1].split(':')[5].split('@')[-1].split(',')
        if name != '<Method_name>':
            third_method_names_dictionary.setdefault(name, '')
            third_method_names_dictionary[name] = tokens
            third_method_names.append(name)
print(len(third_method_names))
print(len(third_method_names_dictionary))

In [ ]:
total_names = []
names1 = []
names2 = []
for name in self_compiled_method_names:
    total_names.append(name)
    names1.append(name)
for name in third_method_names:
    total_names.append(name)
    names2.append(name)
print(len(total_names), len(names1), len(names2))
print(names1[:100])

In [ ]:
# filter out low-quality names
for name in names1:
    if should_trim_name(name):
#         print(name)
        names1 = list(filter((name).__ne__, names1))

for name in names2:
    if should_trim_name(name):
        names2 = list(filter((name).__ne__, names2))
print(len(names1), len(names2))

In [ ]:
def should_trim_name(name):
    if name.find('$') != -1 and len(name.split('$')) <= 2 and not name.startswith('$'):
        return True
    if name == 'main' or name == '<init>' or name == '<clinit>':
        return True
    if name.find('<init>') != -1 or name.find('<clinit>') != -1:
        return True
    if name.find('example') != -1 or name.find('sample') != -1 or name.find('template') != -1:
        return True
    if not any(c.isalpha() for c in name):
        return True
    else:
        return False

In [ ]:
#  only lemmatize verbs
def spacy_lemmatize(tokens, restrict_level=2):
    '''
    restrict_level = 0 : all lemmatize
    restrict_level = 1 : just ignore NOUNs
    restrict_level = 2 : only verbs
    '''
    res = []
    from spacy.lemmatizer import Lemmatizer
    from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
    lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
    nlp = spacy.load("en")
    tokens = ' '.join(tokens)
    doc = nlp(tokens)
    if restrict_level == 0:
        for token in doc:
            pos = token.pos_
            r = lemmatizer(str(token), pos)
            res.append(r[0])
            
    if restrict_level == 1:
        for token in doc:
            pos = token.pos_
#             print(pos)
            if pos != 'NOUN':
                r = lemmatizer(str(token), pos)
                res.append(r[0])
            else:
                res.append(str(token))
                
    if restrict_level == 2:
        for token in doc:
            pos = token.pos_
#             print(pos)
            if pos == 'VERB':
                r = lemmatizer(str(token), pos)
                res.append(r[0])
            else:
                res.append(str(token))
    return res

In [ ]:
# tokenize camel case method names
def camel_split(input):
    res = []
    # <init>, <clinit>
    input = input.strip('<').strip('>')
    # $
    tokens = input.split('$')
    # camel case
    for t in tokens:
        regex = re.compile('[^a-zA-Z]')
        regex.sub('', t)
        t_u = ''
        for _s_ in t:
            if isinstance(_s_, str):
                t_u += _s_ if _s_.islower() else '_' + _s_.lower()
        l = t_u.split('_')
        for r in l:
            if r != '' and not r.isdigit():
                res.append(r)
    return res

In [ ]:
# tokenize names in lower case
def ninja_split(input):
    return wordninja.split(input)

In [ ]:
# filter out names that is not alphabetic
def is_valid_token(input):
    if not any(c.isalpha() for c in input):
        return False
    if input[0].isdigit():
        return False
    if len(input) < 2:
        return False
    if len(set([c for c in input])) < 2 :
        return False
    if input == 'iz':
        return False
    if any(x.isupper() for x in input):
        return False
    return True

In [ ]:
# filter out non-aplhabetic names and names with count < 2
def is_valid_token_with_count(input, counter, count_thr):
    if not any(c.isalpha() for c in input):
        return False
    if input[0].isdigit():
        return False
    if len(input) < 2:
        return False
    if len(set([c for c in input])) < 2 :
        return False
    if counter[input] <= count_thr:
        return False
    if input == 'iz':
        return False
    if any(x.isupper() for x in input):
        return False
    return True

In [ ]:
def should_ninjia(l):
    if l.endswith('i') or l.endswith('j') or l.endswith('k') or l.endswith('t') or l.endswith('s'):
        return True
    if l.startswith('get') or l.startswith('set') or l.startswith('do') or l.startswith('start'):
        return True
    if len(l) > 10:
        return True
    return False

In [ ]:
def should_lemma(l):
    if len(l) < 4:
        return False
    return True


In [ ]:
# remove non alphabetic letters
def refine_letters(input):
    return re.sub(r'[^a-zA-Z]', '', input)

In [ ]:
# tokenize after camel case split
def split_twice(l):
    tokens = []
    tmp = []
    if need_reserve(l):
        tmp.append(l)
    else:
        if should_ninjia(l):
            tokens = ninja_split(l)
        else:
            tokens.append(l)
        for t in tokens:
            if not should_lemma(t):
                tokens = list(filter((t).__ne__, tokens))
                tmp.append(t)
#     print(tmp)
    tokens = spacy_lemmatize(tokens, restrict_level=1)
    tokens.extend(tmp)
    return tokens

In [ ]:
def can_lookup_wiki2vec(t, nl_model):
    try:
        nl_model.get_word_vector(t)
        return True
    except:
        return False

In [ ]:
# get all labels
count = 1
t1 = time.time()
all_labels = []
name_len_dict = {}
with open('/home/qwe/disk1/build_labels/corpus-30w/all_labels.txt', 'w') as f1:
    with open('/home/qwe/disk1/build_labels/corpus-30w/labels1.txt', 'w') as f2:
        with open('/home/qwe/disk1/build_labels/corpus-30w/labels2.txt', 'w') as f3:
            for name in names1:
                name = re.sub('[0-9]', '', name)
                tokens = []
                # INIT_BITSET, SET$ONE
                if all(x.isupper() for x in name.replace('_', '').replace('$', '')):
                    name = name.lower()
                # mCOLLECTION
                if name[0].islower() and all(x.isupper() for x in name[1:].replace('_', '').replace('$', '')):
                    name = name[1:].lower()
                
                _tokens_ = camel_split(name)
                reserve_list = []
                for t in _tokens_:
                    if need_reserve(t):
                        reserve_list.append(t)
                    else:
                        for t2 in ninja_split(t):
                            tokens.append(t2)
                tokens.extend(reserve_list)
                for t in tokens:
                    if not is_valid_token(t):
                        tokens = list(filter((t).__ne__, tokens))
                    
                name_len_dict.setdefault(len(tokens), [])
                name_len_dict[len(tokens)].append(name)
                if len(tokens) < 6 and len(tokens) > 0: 
                    f2.write(name + ': ')
                    f2.write(','.join(tokens))
                    f2.write('\n')
                    for t in tokens:
                        all_labels.append(t)
            
            for name in names2:
                name = re.sub('[0-9]', '', name)
                tokens = []
                if all(x.isupper() for x in name.replace('_', '').replace('$', '')):
                    name = name.lower()
                    
                # mCOLLECTION
                if name[0].islower() and all(x.isupper() for x in name[1:].replace('_', '').replace('$', '')):
                    name = name[1:].lower()
                
                _tokens_ = camel_split(name)
                reserve_list = []
                for t in _tokens_:
                    if need_reserve(t):
                        reserve_list.append(t)
                    else:
                        for t2 in ninja_split(t):
                            tokens.append(t2)
                tokens.extend(reserve_list)
                for t in tokens:
                    if not is_valid_token(t):
                        tokens = list(filter((t).__ne__, tokens))
                
                name_len_dict.setdefault(len(tokens), [])
                name_len_dict[len(tokens)].append(name)
                if len(tokens) < 6 and len(tokens) > 0:
                    f3.write(name + ': ')
                    f3.write(','.join(tokens))
                    f3.write('\n')
                    for t in tokens:
                        all_labels.append(t)
                    
            for t in all_labels:
                f1.write(t)
                f1.write('\n')

In [ ]:
counter = collections.Counter(all_labels)
print(len(counter))

In [ ]:
change = {}
count = 1
for l in set(all_labels):
    if count % 100 == 0:
        print(count)
    before = l
    change.setdefault(before, [])
    after = split_twice(l)
    after = [refine_letters(l) for l in after]
    after = [heuristic_rule(l) for l in after]
    after = [unify_cases(l) for l in after]
    change[before] = after
    count += 1

In [ ]:
refined_labels = []
for l in all_labels:
    ref = change[l]
    for r in ref:
        refined_labels.append(r)
new_counter = collections.Counter(refined_labels)
print(len(refined_labels))
print(len(new_counter))

In [ ]:
result_map = {}
for k, v in change.items():
    result_map.setdefault(k, [])
    for t in v:
        if is_valid_token_with_count(t, new_counter, count_thr=2):
            t = unify_cases(t)
            result_map[k].append(t)

In [ ]:
nl_model = Wikipedia2Vec.load('/home/qwe/disk1/models/wiki2vec/enwiki_20180420_300d.pkl')

In [ ]:
# get final labels
result_labels= []
with open('/home/qwe/disk1/build_labels/corpus-30w/labels1.txt', 'r') as f1:
    with open('/home/qwe/disk1/build_labels/corpus-30w/refined_labels1.txt', 'w') as f2:
        for line in f1.readlines():
            name = line.split(':')[0].strip()
            tokens = line.split(':')[1].strip().split(',')
            refined_tokens = []
            for t in tokens:
                for s in result_map[t]:
                    if can_lookup_wiki2vec(s, nl_model):
                        refined_tokens.append(s)
                    else:
                        refined_tokens.append('<UNK>')
            if len(refined_tokens) > 0:
                f2.write(name + ': ')
                f2.write(','.join(refined_tokens))
                f2.write('\n')
            for t in refined_tokens:
                result_labels.append(t)
                
with open('/home/qwe/disk1/build_labels/data-20w/labels2.txt', 'r') as f1:
    with open('/home/qwe/disk1/build_labels/data-20w/refined_labels2.txt', 'w') as f2:
        for line in f1.readlines():
            name = line.split(':')[0].strip()
            tokens = line.split(':')[1].strip().split(',')
            refined_tokens = []
            for t in tokens:
                for s in result_map[t]:
                    if can_lookup_wiki2vec(s, nl_model):
                        refined_tokens.append(s)
                    else:
                        refined_tokens.append('<UNK>')
            if len(refined_tokens) > 0:
                f2.write(name + ': ')
                f2.write(','.join(refined_tokens))
                f2.write('\n')
            for t in refined_tokens:
                result_labels.append(t)
print(len(result_labels))
print(len(set(result_labels)))

In [ ]:
# a dict to track tokenization
name2tokens = {}
with open('/home/qwe/disk1/build_labels/corpus-30w/refined_labels1.txt', 'r') as f:
    for line in f.readlines():
        name = line.split(': ')[0].strip()
        tokens = line.split(': ')[-1].strip().split(',')
        name2tokens.setdefault(name, None)
        name2tokens[name] = tokens
with open('/home/qwe/disk1/build_labels/corpus-30w/refined_labels2.txt', 'r') as f:
    for line in f.readlines():
        name = line.split(': ')[0].strip()
        tokens = line.split(': ')[-1].strip().split(',')
        name2tokens.setdefault(name, None)
        name2tokens[name] = tokens

In [ ]:
# get all unique labels , add special labels
r2 = list(set(result_labels))
r2.insert(0, '<PAD>')
r2.insert(1, '<SOS>')
r2.insert(2, '<EOS>')
r2.remove('<UNK>')
r2.insert(3, '<UNK>')
print(len(r2))
print(len(set(r2)))
print(r2[:10])

In [ ]:
label_index = {}
index_label = {}
for i, label in enumerate(r2):
    label_index.setdefault(label, '')
    label_index[label] = i
    index_label.setdefault(i, '')
    index_label[i] = label
print(len(label_index), len(index_label))

In [ ]:
# write down all labels
with open('/home/qwe/disk1/data/method-name-predicttion/training-corpus-08.11/records/tokens.csv', 'w') as f:
    for i in sorted(index_label.keys()):
        token = index_label[i]
        print(i, token)
        v = None
        if token == '<PAD>':
            v = np.zeros(300)
        elif token == '<SOS>':
            v = np.ones(300)
        elif token == '<EOS>':
            v = -np.ones(300)
        elif token == '<UNK>':
            v = nl_model.get_word_vector('unk')
        else:
            v = nl_model.get_word_vector(token)
        v = [str(s) for s in v]
#         f.write(str(i) + ': ')
        f.write(','.join(v))
        f.write('\n')

In [ ]:
# write down tokens names
with open('/home/qwe/disk1/data/method-name-predicttion/training-corpus-08.11/records/token_names.txt', 'w') as f:
    for k, v in label_index.items():
        f.write(str(k) + '\t' + str(v))
        f.write('\n')

In [ ]:
# get id : full dictionary
id2full = {}
with open('/home/qwe/disk1/data_SoC/files/IDs.txt', 'r') as f:
    for line in f.readlines():
        full = line.split(':::')[0]
        id = int(line.split(':::')[-1])
        id2full.setdefault(id, None)
        id2full[id] = full
len(id2full)

In [ ]:
# get valid id list and write to file
valid_id_list = []
with open('/home/qwe/disk1/data/method-name-predicttion/training-corpus-08.11/train/valid_id.txt', 'w') as f:
    for i in sorted(id2full.keys()):
        full = id2full[i]
        name = full2simple[full]
        if name2tokens.get(name) is not None:
            valid_id_list.append(i)
            f.write(str(i))
            f.write('\n')
len(valid_id_list)         